<h1 style="font-family: 'poppins'; font-weight: bold; color: white;">Twitter Sentiment Analysis</h1>
<h3 style="font-family: 'poppins'; font-weight: bold; color: white;">👨‍💻Author: Hasnain Yaqoob</h3>

[![GitHub](https://img.shields.io/badge/GitHub-Profile-blue?style=for-the-badge&logo=github)](https://github.com/hasnainyaqub)
[![Kaggle](https://img.shields.io/badge/Kaggle-Profile-blue?style=for-the-badge&logo=kaggle)](https://www.kaggle.com/hasnainyaqooob)
[![LinkedIn](https://img.shields.io/badge/LinkedIn-Profile-blue?style=for-the-badge&logo=linkedin)](https://www.linkedin.com/in/hasnain-yaqoob-/)  


[![Twitter/X](https://img.shields.io/badge/Twitter-Profile-blue?style=for-the-badge&logo=twitter)](https://twitter.com/Hasnain_Yaqoob_)
[![Facebook](https://img.shields.io/badge/Facebook-Profile-blue?style=for-the-badge&logo=facebook)](https://www.facebook.com/hasnainyaqkoob)
[![TikTok](https://img.shields.io/badge/TikTok-Profile-black?style=for-the-badge&logo=tiktok)](https://www.tiktok.com/@hasnainyaqub)  

[![Instagram](https://img.shields.io/badge/Instagram-Profile-blue?style=for-the-badge&logo=instagram)](https://www.instagram.com/hasnain_yaqub/)
[![Email](https://img.shields.io/badge/Email-Contact%20Me-red?style=for-the-badge&logo=email)](mailto:hasnainyaqubakm@gmail.com)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional,GRU,SimpleRNN,Dense,Dropout,BatchNormalization,LSTM,InputLayer,Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
# import data
df = pd.read_csv('data/clean_df.csv')
dfv = pd.read_csv('data/clean_dfv.csv')

2025-11-12 13:12:18.203883: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-12 13:12:18.248906: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-12 13:12:19.429363: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
df.dropna(inplace=True)

In [4]:
# create mapping dict for sentiments to integers (Positive:0, Negative:1, Neutral:2, irrelevant:3)
map_dict={val:idx for idx,val in enumerate(df["Sentiment"].unique())}
# split data into train and val
X_train = df['clean_text'].values
y_train = df['Sentiment'].map(map_dict)

X_val = dfv['clean_text'].values
y_val = dfv['Sentiment'].map(map_dict)

In [5]:
lengths = np.array([len(str(x).split()) for x in X_train])
print("Max length:", lengths.max())
print("Average length:", lengths.mean())
print("90th percentile:", np.percentile(lengths, 90))
print("95th percentile:", np.percentile(lengths, 95))


Max length: 99
Average length: 9.107430667481799
90th percentile: 19.0
95th percentile: 22.0


In [6]:
MAX_LEN = 100  
# Tokenizer 
tokenizer = Tokenizer( oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
# Convert texts to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN)
X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN)
# let create vocab size
vocab=len(tokenizer.word_index)+1

In [45]:
import pickle

# Save
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# Simple RNN 

In [46]:
# Simple RNN
rnn_model = Sequential()
rnn_model.add(InputLayer(input_shape=(100,)))
rnn_model.add(Embedding(input_dim=vocab, output_dim=64))
rnn_model.add(SimpleRNN(128, activation="tanh", return_sequences=True))  # <--- return full sequence
rnn_model.add(SimpleRNN(64, activation="tanh"))  # second RNN processes the sequence
rnn_model.add(Dense(4, activation="softmax"))

# Compile the model
rnn_model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
# Print the model summary
rnn_model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)        │ (None, 100, 64)        │     1,392,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_8 (SimpleRNN)        │ (None, 100, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_9 (SimpleRNN)        │ (None, 64)             │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,429,956 (5.45 MB)

 Trainable params: 1,429,956 (5.45 MB)

 Non-trainable params: 0 (0.00 B)

In [47]:
# Define early stopping callback
callbacks=tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                       patience=3,
                                       verbose=True,
                                       min_delta=0.0001)

In [48]:
# Check the shapes of the padded sequences and labels
print(X_train_pad.shape)
print(y_train.shape)
print(X_val_pad.shape)
print(y_val.shape)

(71972, 100)
(71972,)
(1000, 100)
(1000,)


In [49]:
print(len(X_train_pad), len(y_train))
print(len(X_val_pad), len(y_val))

71972 71972
1000 1000


In [ ]:
# Save model after every epoch
checkpoint = ModelCheckpoint(
    "dl_models/simple_Rnn_epoch_{epoch:02d}.keras",  # filename pattern
    save_weights_only=False,     # save full model (architecture + weights)
    save_freq='epoch',           # save every epoch
    verbose=1
)

# Training with checkpoint and early stopping
history = rnn_model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=10,
    batch_size=64,
    callbacks=[callbacks, checkpoint]  # add both early stopping and checkpoint
)

Epoch 1/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.4748 - loss: 1.1489
Epoch 1: saving model to dl_models/gru_epoch_01.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 38s 33ms/step - accuracy: 0.6231 - loss: 0.9025 - val_accuracy: 0.8680 - val_loss: 0.4042
Epoch 2/10
1124/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8767 - loss: 0.3491
Epoch 2: saving model to dl_models/gru_epoch_02.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 40s 35ms/step - accuracy: 0.8741 - loss: 0.3541 - val_accuracy: 0.8740 - val_loss: 0.3778
Epoch 3/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9270 - loss: 0.2113
Epoch 3: saving model to dl_models/gru_epoch_03.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 40s 36ms/step - accuracy: 0.9194 - loss: 0.2284 - val_accuracy: 0.8860 - val_loss: 0.4360
Epoch 4/10
1124/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9336 - loss: 0.1886
Epoch 4: saving model to dl_models/gru_epoch_04.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 40s 35ms/step - accuracy: 0.9298 -

# GRU

In [ ]:
# Make sure vocab and MAX_LEN are defined
vocab = len(tokenizer.word_index) + 1
MAX_LEN = 100  # or whatever length you padded your sequences to

gru_model = Sequential()

# Input
gru_model.add(InputLayer(input_shape=(MAX_LEN,)))

# Embedding
gru_model.add(Embedding(input_dim=vocab, output_dim=64))

# First GRU layer (returns full sequence)
gru_model.add(GRU(128, activation="tanh", return_sequences=True))

# Second GRU layer (returns last hidden state)
gru_model.add(GRU(64, activation="tanh"))

# Dense output for 4-class classification
gru_model.add(Dense(4, activation="softmax"))

# Compile model
gru_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Early stopping callback
callbacks = EarlyStopping(
    monitor="val_loss",
    patience=3,
    verbose=1,
    restore_best_weights=True
)

# Model summary
gru_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 100, 64)        │     1,392,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, 100, 128)       │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_6 (GRU)                     │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,504,644 (5.74 MB)

 Trainable params: 1,504,644 (5.74 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
# Save model after every epoch
checkpoint = ModelCheckpoint(
    "dl_models/gru_epoch_{epoch:02d}.keras",  # filename pattern
    save_weights_only=False,     # save full model (architecture + weights)
    save_freq='epoch',           # save every epoch
    verbose=1
)

# Training with checkpoint and early stopping
history = gru_model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=10,
    batch_size=64,
    callbacks=[callbacks, checkpoint]  # add both early stopping and checkpoint
)

Epoch 1/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.9259 - loss: 0.2105
Epoch 1: saving model to dl_models/gru_epoch_01.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 114s 102ms/step - accuracy: 0.9217 - loss: 0.2194 - val_accuracy: 0.9170 - val_loss: 0.3299
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.9381 - loss: 0.1743
Epoch 2: saving model to dl_models/gru_epoch_02.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 121s 108ms/step - accuracy: 0.9343 - loss: 0.1848 - val_accuracy: 0.9160 - val_loss: 0.3635
Epoch 3/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.9452 - loss: 0.1563
Epoch 3: saving model to dl_models/gru_epoch_03.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 120s 107ms/step - accuracy: 0.9422 - loss: 0.1626 - val_accuracy: 0.9160 - val_loss: 0.3478
Epoch 4/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.9493 - loss: 0.1406
Epoch 4: saving model to dl_models/gru_epoch_04.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 131s 116ms/step - accura

In [53]:
# plot the training and validation accuracy and loss in a single plotly 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=2, subplot_titles=("Accuracy", "Loss"))
fig.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines+markers', name='Training Accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines+markers', name='Validation Accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines+markers', name='Training Loss'), row=1, col=2)
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines+markers', name='Validation Loss'), row=1, col=2)
fig.update_layout(title_text="Training and Validation Metrics", showlegend=True)
fig.show()

# GRU with DropOut

In [8]:
# vocab and MAX_LEN 
vocab = len(tokenizer.word_index) + 1
MAX_LEN = 100  # adjust to your padded sequence length

dr_gru_model = Sequential()

# Input
dr_gru_model.add(InputLayer(input_shape=(MAX_LEN,)))

# Embedding
dr_gru_model.add(Embedding(input_dim=vocab, output_dim=64))

# First GRU layer (return full sequence) + dropout
dr_gru_model.add(GRU(128, activation="tanh", return_sequences=True))
dr_gru_model.add(Dropout(0.2))

# Second GRU layer (return last hidden state) + dropout
dr_gru_model.add(GRU(64, activation="tanh"))
dr_gru_model.add(Dropout(0.2))

# Dense output layer for 4-class sentiment
dr_gru_model.add(Dense(4, activation="softmax"))

# Compile
dr_gru_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Early stopping
dr_callbacks = EarlyStopping(
    monitor="val_loss",
    patience=3,
    verbose=1,
    restore_best_weights=True
)

# Summary
dr_gru_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 100, 64)        │     1,392,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 100, 128)       │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,504,644 (5.74 MB)

 Trainable params: 1,504,644 (5.74 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Save model after every epoch
dr_cp = ModelCheckpoint(
    "dl_models/dr_gru_epoch_{epoch:02d}.keras",  # filename pattern
    save_weights_only=False,     # save full model (architecture + weights)
    save_freq='epoch',           # save every epoch
    verbose=1
)

# Training with checkpoint and early stopping
history = dr_gru_model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=10,
    batch_size=64,
    callbacks=[dr_callbacks, dr_cp]  # add both early stopping and checkpoint
)

Epoch 1/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.5569 - loss: 1.0438
Epoch 1: saving model to dl_models/dr_gru_epoch_01.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 119s 104ms/step - accuracy: 0.6627 - loss: 0.8503 - val_accuracy: 0.8550 - val_loss: 0.4131
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.8391 - loss: 0.4535
Epoch 2: saving model to dl_models/dr_gru_epoch_02.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 118s 105ms/step - accuracy: 0.8428 - loss: 0.4370 - val_accuracy: 0.9050 - val_loss: 0.3082
Epoch 3/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.8923 - loss: 0.2992
Epoch 3: saving model to dl_models/dr_gru_epoch_03.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 114s 102ms/step - accuracy: 0.8877 - loss: 0.3106 - val_accuracy: 0.9100 - val_loss: 0.3037
Epoch 4/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.9124 - loss: 0.2423
Epoch 4: saving model to dl_models/dr_gru_epoch_04.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 116s 103ms/s

In [11]:
# plot the training and validation accuracy and loss in a single plotly 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=2, subplot_titles=("Accuracy", "Loss"))
fig.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines+markers', name='Training Accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines+markers', name='Validation Accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines+markers', name='Training Loss'), row=1, col=2)
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines+markers', name='Validation Loss'), row=1, col=2)
fig.update_layout(title_text="Training and Validation Metrics", showlegend=True)
fig.show()

# Bidirectional GRU

In [12]:
bigru_model = Sequential()

# Input
bigru_model.add(InputLayer(input_shape=(MAX_LEN,)))

# Embedding
bigru_model.add(Embedding(input_dim=vocab, output_dim=64))

# First Bidirectional GRU layer (return full sequence) + dropout
bigru_model.add(Bidirectional(GRU(128, activation="tanh", return_sequences=True)))
bigru_model.add(Dropout(0.2))

# Second Bidirectional GRU layer (return last hidden state) + dropout
bigru_model.add(Bidirectional(GRU(64, activation="tanh")))
bigru_model.add(Dropout(0.2))

# Dense output layer for 4-class sentiment
bigru_model.add(Dense(4, activation="softmax"))

# Compile
bigru_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Early stopping
callbacks = EarlyStopping(
    monitor="val_loss",
    patience=3,
    verbose=1,
    restore_best_weights=True
)

# Model summary
bigru_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 64)        │     1,392,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 256)       │       148,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │       123,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,665,796 (6.35 MB)

 Trainable params: 1,665,796 (6.35 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Save model after every epoch
bi_cp = ModelCheckpoint(
    "dl_models/bi_gru_epoch_{epoch:02d}.keras",  # filename pattern
    save_weights_only=False,     # save full model (architecture + weights)
    save_freq='epoch',           # save every epoch
    verbose=1
)
# Training with checkpoint and early stopping
history = bigru_model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=10,
    batch_size=64,
    callbacks=[callbacks, bi_cp]  # add both early stopping and checkpoint
)

Epoch 1/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.5534 - loss: 1.0424
Epoch 1: saving model to dl_models/bi_gru_epoch_01.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 159s 139ms/step - accuracy: 0.6666 - loss: 0.8407 - val_accuracy: 0.8760 - val_loss: 0.3989
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.8474 - loss: 0.4260
Epoch 2: saving model to dl_models/bi_gru_epoch_02.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 153s 136ms/step - accuracy: 0.8501 - loss: 0.4178 - val_accuracy: 0.9000 - val_loss: 0.3031
Epoch 3/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.8943 - loss: 0.2951
Epoch 3: saving model to dl_models/bi_gru_epoch_03.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 153s 136ms/step - accuracy: 0.8925 - loss: 0.3023 - val_accuracy: 0.9080 - val_loss: 0.2944
Epoch 4/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.9164 - loss: 0.2375
Epoch 4: saving model to dl_models/bi_gru_epoch_04.keras
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 153s 136ms/s

In [14]:
# plot the training and validation accuracy and loss in a single plotly 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=2, subplot_titles=("Accuracy", "Loss"))
fig.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines+markers', name='Training Accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines+markers', name='Validation Accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines+markers', name='Training Loss'), row=1, col=2)
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines+markers', name='Validation Loss'), row=1, col=2)
fig.update_layout(title_text="Training and Validation Metrics", showlegend=True)
fig.show()